In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tools import BaseK,show_plot,Solver,Differ,Comparator
from matplotlib.ticker import FuncFormatter
from matplotlib.widgets import Slider, Button
import copy

In [2]:
%matplotlib qt

In [3]:
class K(BaseK):
    light = 0.0001
    et = 1e9  # 8 10
    et_ = 1e6  # 5 6
    e = 1e9  # 8 10
    e_ = 1e8  # 7 9

    H = 1e9 # 8 10

    r = 1e9
    p = 0.001
    qH = 2000
    disp = 1e9
    ph = 1e-5
    s = 2
    d = 0.05
    c = 1
    rD = 1e9

In [10]:
T = np.linspace(0, 0.000_1, 100)

In [5]:
def system_full(t, C, k=K()):
    [Q, DH, QHH, D, Qt, QHD, QH, QD, Q_DH] = C

    ret = k.et*k.light * Q
    ret_ = k.et_ * Qt

    re = k.e * Qt * DH
    re_ = k.e_ * Q_DH

    rH = k.H * Q_DH

    rqH = k.qH * Qt * QHH
    rph = k.ph * Qt
    rs = k.s * Qt * QHD

    rc = k.c * Qt * D
    rd = k.d * QD

    rc = k.r * QH * D


    rp = k.p * QHD
    rdisp = k.disp * QH * QH

    rrD = k.rD * D * D

    Q = -ret + ret_ + rdisp + rd
    Qt = ret - ret_ - re - rqH - rph - rs - rc + re_
    DH = -re + re_
    Q_DH = re - re_ - rH
    QHH = -rqH + rp + rdisp
    QH = 2 * rqH + rs + rH - rc - 2 * rdisp
    QHD = -rs + rc - rp
    QD = rs + rc - rd
    D = -rc + rH - rc + rd - 2 * rrD
    return [Q, DH, QHH, D, Qt, QHD, QH, QD, Q_DH]


c_full = '[Q, DH, QHH, D, Qt, QHD, QH, QD, Q_DH]'
initial_full = [1, 1, 0, 0, 0, 0, 0, 0, 0]

In [6]:
def system_ions(t, C, k=K()):
    [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp] = C
    re = k.e * k.light * Q * DH
    rH = k.H * Qm * DHp
    rr = k.r * QH * D
    rp = k.p * QHD
    rqH = k.qH * Q * QHH
    rdisp = k.disp * QH * QH
    rph = k.ph * Q
    rs = k.s * Q * QHD
    rd = k.d * QD
    rc = k.c * Q * D
    rrD = k.rD * D * D

    Q = -re - rqH - rph - rs - rc + rdisp + rd
    DH = -re
    QHH = -rqH + rp + rdisp
    D = -rr - rc - 2 * rrD + rH + rd
    QHD = -rp - rs + rr
    QH = -rr - 2 * rdisp + rH + 2 * rqH + rs
    QD = -rd + rs + rc
    Qm = -rH + re
    DHp = -rH + re

    return [Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]


c_ions = '[Q, DH, QHH, D, QHD, QH, QD, Qm, DHp]'
initial_ions = [1, 1, 0, 0, 0, 0, 0, 0, 0]

In [21]:
def system_1(t, C, k=K()):
    A, B = C

    ra = k.et * A*1e-4
    rb = k.et * B*1e-4
    A = -ra
    B = ra
    return [A, B]


def system_2(t, C, k=K()):
    A, B = C

    ra = k.et * A*1e-4
    rb = k.et_ * B*1e-4*1e3
    A = -ra + rb
    B = rb - rb
    return [A, B]


c0 = '[A, B]'
init0 = [1, 1]

#   Plot

In [22]:
# Y= Solver(system_ions, K(), initial_ions, c_ions, T)
# y = Solver(system_full, K(), initial_full, c_full, T)
# Y = Differ(
#     Solver(system_full, K(), initial_full, c_full, T),
#     Solver(system_ions, K(), initial_ions, c_ions, T),
# )
Y = Differ(
    Solver(system_1, K(), init0, c0, T),
    Solver(system_2, K(), init0, c0, T),
)
SET_LIM = False
AUTO = True

In [25]:
# Plots
lims = 50
fig, ax = plt.subplots()
gs = plt.GridSpec(2, 2, figure=fig)
fig.subplots_adjust(left=0.25, right=0.99, bottom=0.2, top=0.95, hspace=0.1, wspace=0.1)
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['bottom'].set_visible(False)
ax.spines['left'].set_visible(False)

# Plots
scal = FuncFormatter(lambda x, pos: f'{x*1000: .2f}')
K_base = Y.K
Y.solve()

plots = {}
base_ax = plt.subplot(gs[0, 0:2])
for c in ['A', 'B']:
    plots[c] = base_ax.plot(Y.T, Y[c], label=c)[0]
base_ax.legend()
base_ax.xaxis.set_major_formatter(scal)
if SET_LIM:
    base_ax.set_ylim([-lims, lims])

# base_ax = plt.subplot(gs[0, 0:2])
# for c in ['Q', 'DH', 'QHH']:
#     plots[c] = base_ax.plot(Y.T, Y[c], label=c)[0]
# base_ax.legend()
# base_ax.xaxis.set_major_formatter(scal)
# if SET_LIM:
#     base_ax.set_ylim([-lims, lims])

# d_ax = plt.subplot(gs[1, 0])
# plots['D'] = d_ax.plot(Y.T, Y['D'], label='D')[0]
# d_ax.legend()
# d_ax.xaxis.set_major_formatter(scal)
# if SET_LIM:
#     d_ax.set_ylim([-lims, lims])

# other_ax = plt.subplot(gs[1, 1])
# other_ax.xaxis.set_major_formatter(scal)
# for c in ['QH', 'QHD']:
#     plots[c] = other_ax.plot(Y.T, Y[c], label=c)[0]
# other_ax.legend()
# other_ax.xaxis.set_major_formatter(scal)
# if SET_LIM:
#     other_ax.set_ylim([-lims, lims])


# Sliders
sliders = {}
buttons = {}
K_new = copy.deepcopy(K_base)


def resolve(event=None):
    Y.solve(K=K_new)
    for c, plot in plots.items():
        plot.set_ydata(Y[c])
    fig.canvas.draw_idle()


def get_slider_action(k):
    def update(val):
        K_new[k] = K_base[k] * 10**val
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        if AUTO:
            resolve()

    return update


def get_zero_button_action(k):
    def update(event):
        K_new[k] = 0
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
        fig.canvas.draw_idle()
        if AUTO:
            resolve()

    return update


def reset(event):
    global K_new
    K_new = copy.deepcopy(K_base)
    for k in sliders:
        sliders[k].reset()
        buttons[k].label.set_text(f"{K_new[k]: .1e}")
    fig.canvas.draw_idle()
    if AUTO:
        resolve()


solve_axes = fig.add_axes([0.02, 0.05, 0.05, 0.05])
solve_button = Button(solve_axes, 'Solve', hovercolor='0.975')
solve_button.on_clicked(resolve)

reset_axes = fig.add_axes([0.08, 0.05, 0.05, 0.05])
reset_button = Button(reset_axes, 'Reset', hovercolor='0.975')
reset_button.on_clicked(reset)

for i, k in enumerate(K_base):
    k_axes = fig.add_axes(
        [
            0.03,  # left
            0.95 - 0.04 * i,  # bottom
            0.10,  # width
            0.03,  # height
        ]
    )
    amp_slider = Slider(
        ax=k_axes,
        label=k,
        valmin=-3,
        valmax=3,
        valinit=0,
        valstep=0.1,
        orientation="horizontal",
    )
    sliders[k] = amp_slider
    amp_slider.on_changed(get_slider_action(k))


    k_zero_axes = fig.add_axes(
        [
            0.15,  # left
            0.955 - 0.04 * i,  # bottom
            0.05,  # width
            0.02,  # height
        ]
    )
    zero_button = Button(k_zero_axes, f"{K_new[k]: .1e}", hovercolor='0.975')
    buttons[k] = zero_button
    zero_button.on_clicked(get_zero_button_action(k))


In [49]:
zero_button.label.set_text('1')

In [ ]:
def system_activated(t, C, k=K):
    [Qt,Qs, DH, QHH, D, QHD, QH, QD] = C
    Qs = -R1
    Qt = R1 - R2 - R3 - R4 - R5
    DH = -R2
    QH = R2 + 2*R3 + R4 - R6 - 2*R7
    D = R2 - R5 - R6 + R8 - 2*R10
    QHH = -R3 + R7 + R9
    QHD = -R4 + R6 - R9
    QD = R4 + R5 - R8
    Q = R7 + R8 - R11

    re = k['Ke'] * k['light'] * Q * DH
    rr = k['Kr'] * QH * D
    rp = k['Kp'] * QHD
    rqH = k['KqH'] * Q * QHH
    rdisp = k['Kdisp'] * QH * QH
    rph = k['Kph'] * Q
    rs = k['Ks'] * Q * QHD
    rd = k['Kd'] * QD
    rc = k['Kc'] * Q * D
    rrD = k['KrD'] * D * D

    R1 = k['e']*k['light']*Qs
    R2 = k['Ke']*Qt*DH
    R3 = k['KqH']*Q*QHH
    R4 = k['Ks']*Qt*QHD
    R5 = k['Kr']*Qt*D
    R6 = k['Kr']*QH*D
    R7 = k['Kdisp']*QH*QH
    R8 = k['Kd']*QD
    R9 = k['Kp']*QHD
    R10 = k['KrD']*D*D

    return [Q, DH, QHH, D, QHD, QH, QD]